---
Vérifier qu'on n'est pas en 3.10.2 (sinon PyTorch ne marchera pas)

In [2]:
import sys
print(sys.version)

3.9.10 (tags/v3.9.10:f2f3f53, Jan 17 2022, 15:14:21) [MSC v.1929 64 bit (AMD64)]


---
Imports - notemment de clubs_gym (https://github.com/fschlatt/clubs_gym)

In [12]:
import gym
import clubs_gym
import time
import random

---
Liste des environnements disponibles

In [4]:
clubs_gym.ENVS

['LeducTwoPlayer-v0',
 'KuhnTwoPlayer-v0',
 'KuhnThreePlayer-v0',
 'LimitHoldemTwoPlayer-v0',
 'LimitHoldemSixPlayer-v0',
 'LimitHoldemNinePlayer-v0',
 'NoLimitHoldemTwoPlayer-v0',
 'NoLimitHoldemSixPlayer-v0',
 'NoLimitHoldemNinePlayer-v0',
 'NoLimitHoldemBbAnteNinePlayer-v0',
 'PotLimitOmahaTwoPlayer-v0',
 'PotLimitOmahaSixPlayer-v0',
 'PotLimitOmahaNinePlayer-v0',
 'ShortDeckTwoPlayer-v0',
 'ShortDeckSixPlayer-v0',
 'ShortDeckNinePlayer-v0']

Ne nous intéressera que `NoLimitHoldemTwoPlayer-v0` et `NoLimitHoldemSixPlayer-v0`

---
Ouvrons `NoLimitHoldemTwoPlayer-v0`.

In [5]:
env_str = "NoLimitHoldemTwoPlayer-v0"
num_players = 2  # heads-up
num_streets = 4  # texas hold'em
blinds = [1, 2]  # as on the tables on which we used to play @Jean
antes = 0  # a bit weird to have antes in a heads-up I reckon (single ints are expanded to the number of players so could've inputted a list)
raise_sizes = ['inf', 'inf', 'inf', 'inf']  # yes the documentation is wrong, 'inf' should be used and not float('inf')
num_raises = 'inf'  # max number of bets for each street* (including preflop)
num_suits = 4  # number of suits to use in deck (obviously 4 otherwise I mean it's weird)
num_ranks = 13  # same, obviously 13 (A, 2, 3, 4, 5, 6, 7, 8, 9, T, J, Q, K)
num_hole_cards = 2  # **
mandatory_num_hole_cards = 0  # you can use the fives on the board and it will end up being a draw if your enemy does it too
num_community_cards = [0, 3, 1, 1]  # preflop, flop, turn, river
start_stack = 100  # as on the tables on which we used to play @Jean (actually it was any integer between 80 and 200)
n_cards_for_hand = 5  # texas hold'em
order = ['sf', 'fk', 'fh', 'fl', 'st', 'tk', 'tp', 'pa', 'hc']  # most likely optional, but just to make sure we're working with the right order

config_dict = {'num_players': num_players,
               'num_streets': num_streets,
               'blinds': blinds,
               'antes': antes,
               'raise_sizes': raise_sizes,
               'num_raises': num_raises,
               'num_suits': num_suits,
               'num_ranks': num_ranks,
               'num_hole_cards': num_hole_cards,
               'mandatory_num_hole_cards': mandatory_num_hole_cards,
               'num_community_cards': num_community_cards,
               'start_stack': start_stack,
               'num_cards_for_hand': n_cards_for_hand,
               'order': order}

clubs_gym.envs.register({env_str: config_dict})
env = gym.make(env_str)

C:\Users\lhott\Documents\Formation scolaire\These\Formations\RL\Projet\reinforcement-learning-poker\venv\lib\site-packages\gym\envs\registration.py:595: UserWarning: WARN: Overriding environment NoLimitHoldemTwoPlayer-v0
  logger.warn(f"Overriding environment {id}")


---
Un peu de vocabulaire pour les zguegs (là où y avait des * au-dessus):
- \* A card that’s dealt in a particular betting round, in many poker variants, is called a **street**. For example, in Texas Hold’em and Omaha, the community cards are referred to as the flop, turn and river, with the turn and river sometimes referred to as Fourth Street and Fifth Street.
- ** The term **hole cards' in poker signals that the cards are private cards that should only be viewed by the player. The dealer deals hole cards face down and players should ensure that the value of those cards remains secret.

---
`obs` est illisible donc on fait une fonction `print_obs` pour avoir des jolis prints sur ce qu'il se passe.

In [20]:
def print_obs(obs):
    """
    Fancy print of stats for the clubs gym poker gym.
    All Cards displayed (on voit les cartes de tout le monde quoi)
    
    param: obs: le premier chose renvoyé par env.step
    return: nada
    """
    for _ in range(5):
        print('---')
    print('------------------- Players\' stacks -------------------')
    print(f"player_0: {obs['stacks'][0]}\n" + \
          f"player_1: {obs['stacks'][1]}")
    print('------------------- Game Stats -------------------')
    print(f'community_cards: {obs["community_cards"]}')
    print(f'Dealer/Button: {obs["button"] % 2}')
    print(f'Pot: {obs["pot"]}')
    print(f'Commits: player_0 {obs["street_commits"][0]}, player_1 {obs["street_commits"][1]}')

    if not obs['action'] == -1:
        print(f"\n---------------- player {obs['action']}'s turn ----------------")
        print(f'hole_cards: {obs["hole_cards"]}')
        print(f'call: {obs["call"]}, min_raise: {obs["min_raise"]}, max_raise: {obs["max_raise"]}', end='\n\n')
    else:
        print('\n[INFO] - End of the Game.')

---
Main

Dans `env.step`, mettre un integer qui correspond à ce que vous voulez miser. 
- Si la mise est inférieure à la mise minimum, vous vous couchez. 
- Si la mise est supérieure à la mise maximum (en no limit, ça veut dire vous voulez miser plus que ce que vous détenez) alors all in.

Le truc ci-dessous vous fait changer de joueur pré/post flop - désolé. This is because *in Heads Up poker, the button acts first preflop and last postflop*.

In [31]:
# Start the game

observation = env.reset(reset_stacks = True)
env.render()
print(f"Action space is of size: {env.action_space}")

print("---\nGame started\n---")
counter = 0

# print(f"Hand number {counter + 1}\n---")

print_obs(observation)

while True:
    if counter % 2 == 0: # le bot (player_0) joue de façon random, mais mieux qu'en appelant env.action_space.sample() de façon débile
        counter += 1
        bot_bet = random.randint(0, obs['stacks'][0])
        obs, rewards, done, info = env.step(bot_bet)  # take a random action
        env.render()
        print(f"ID of the action that was taken : {bot_bet}")
        print_obs(obs)

        if all(done):
            break

    else:  # notre tour
        counter +=1
        joueur_bet = input("Combien voulez-vous miser ?")
        obs, rewards, done, info = env.step(int(joueur_bet))
        env.render()
        print(f"Vous avez misé : {joueur_bet}")
        print_obs(obs)
        
        if all(done):
            break


env.close()

Action space is of size: Discrete(200)
---
Game started
---
---
---
---
---
---
------------------- Players' stacks -------------------
player_0: 99
player_1: 98
------------------- Game Stats -------------------
community_cards: []
Dealer/Button: 0
Pot: 3
Commits: player_0 1, player_1 2

---------------- player 0's turn ----------------
hole_cards: [Card (2161628839264): 2♦, Card (2161628814544): 5♦]
call: 1, min_raise: 3, max_raise: 99

ID of the action that was taken : 74
---
---
---
---
---
------------------- Players' stacks -------------------
player_0: 25
player_1: 98
------------------- Game Stats -------------------
community_cards: []
Dealer/Button: 0
Pot: 77
Commits: player_0 75, player_1 2

---------------- player 1's turn ----------------
hole_cards: [Card (2161628523680): 8♠, Card (2161629032944): T♠]
call: 73, min_raise: 98, max_raise: 98



Combien voulez-vous miser ? 0


Vous avez misé : 0
---
---
---
---
---
------------------- Players' stacks -------------------
player_0: 102
player_1: 98
------------------- Game Stats -------------------
community_cards: [Card (2161629032656): 9♥, Card (2161629033904): Q♦, Card (2161090097984): 8♣, Card (2161090098368): 8♦, Card (2161629033520): J♦]
Dealer/Button: 0
Pot: 0
Commits: player_0 0, player_1 0

[INFO] - End of the Game.
ID of the action that was taken : 98
---
---
---
---
---
------------------- Players' stacks -------------------
player_0: 102
player_1: 98
------------------- Game Stats -------------------
community_cards: [Card (2161629032656): 9♥, Card (2161629033904): Q♦, Card (2161090097984): 8♣, Card (2161090098368): 8♦, Card (2161629033520): J♦]
Dealer/Button: 0
Pot: 0
Commits: player_0 0, player_1 0

[INFO] - End of the Game.
